In [ ]:
import timeit
import numpy as np
from sklearn.datasets import make_classification
from sklearn.feature_selection import chi2 as chi2_sklearn
from chi2 import chi2_numba

# Set up the test parameters
N_SAMPLES = 2000
N_FEATURES = 2000
N_CLASSES = 5
RANDOM_STATE = 42

print("Chi-Squared Implementation Benchmark")
print("-" * 40)
print(f"Dataset shape: Samples={N_SAMPLES}, Features={N_FEATURES}, Classes={N_CLASSES}")
print("-" * 40)

# 1. Generate synthetic data
X, y = make_classification(
    n_samples=N_SAMPLES,
    n_features=N_FEATURES,
    n_informative=500,
    n_redundant=500,
    n_classes=N_CLASSES,
    n_clusters_per_class=1,
    random_state=RANDOM_STATE
)
# The Chi-squared test requires non-negative features (e.g., counts)
X = np.abs(X * 100).astype(np.int64)

# 2. Run the Numba implementation
# First run is for JIT compilation ("warm-up") and is not timed.
print("Compiling Numba function...")
chi2_numba(X, y)
print("Compilation complete.\n")

# Time the Numba implementation
print("Timing Numba implementation...")
numba_time = timeit.timeit(lambda: chi2_numba(X, y), number=10)
print(f"Done.")

# 3. Run the scikit-learn implementation
print("\nTiming scikit-learn implementation...")
sklearn_time = timeit.timeit(lambda: chi2_sklearn(X, y), number=10)
print(f"Done.")

# 4. Verify that the results are the same
chi2_n, p_n = chi2_numba(X, y)
chi2_s, p_s = chi2_sklearn(X, y)

assert np.allclose(chi2_n, chi2_s), "Chi2 statistics do not match!"
assert np.allclose(p_n, p_s), "P-values do not match!"
print("\nCorrectness check passed: Results are identical.")

# 5. Report the results
print("\n\n--- Benchmark Results ---")
print(f"Scikit-learn time: {sklearn_time:.4f} seconds")
print(f"Numba time:        {numba_time:.4f} seconds")

speedup = sklearn_time / numba_time
print(f"\nNumba implementation is {speedup:.2f}x faster.")

In [1]:
# final_benchmark_with_feature_engine.py
import os
os.environ['NUMBA_NUM_THREADS'] = '8'
import time
import numpy as np
import pandas as pd
from numba import njit, prange
from math import log
from sklearn.feature_selection import chi2, f_regression, mutual_info_classif, mutual_info_regression
from itertools import combinations
from numba.types import Tuple


import numpy as np
from numba import njit, prange, float32, int32, int64

# Numba type signatures for eager compilation
# This signature assumes X is float32 and y is int32. Adjust if different.
# It reads as: "A function that returns a float32, and takes two float32 arrays,
# their respective number of states (int32), and the number of samples (int64)."
MI_SIGNATURE = float32(float32[:], int32, float32[:], int32, int64)

@njit(MI_SIGNATURE, cache=True, nogil=True)
def _calculate_mi_optimized(x1, n_states1, x2, n_states2, n_samples):
    """
    Calculates Mutual Information between two discrete vectors.
    Uses float32 for better performance and memory usage.
    """
    contingency_table = np.zeros((n_states1, n_states2), dtype=np.float32)

    for i in range(n_samples):
        contingency_table[int(x1[i]), int(x2[i])] += 1

    contingency_table /= n_samples

    p1 = np.sum(contingency_table, axis=1)
    p2 = np.sum(contingency_table, axis=0)
    mi = 0.0

    for i in range(n_states1):
        for j in range(n_states2):
            p_xy = contingency_table[i, j]
            p_x = p1[i]
            p_y = p2[j]
            if p_xy > 1e-12 and p_x > 1e-12 and p_y > 1e-12:
                mi += p_xy * np.log(p_xy / (p_x * p_y))

    return mi

# Note the explicit signature. It defines the return tuple and input types.
# It reads as: "A function returning a tuple of (float32[:], float32[:, ::1])
# that takes a 2D float32 array (C-contiguous) and a 1D int32 array."
PRECOMPUTE_SIGNATURE = Tuple((float32[:], float32[:, ::1]))(float32[:, ::1], int32[:])

@njit(PRECOMPUTE_SIGNATURE, parallel=True, cache=True)
def _precompute_mi_matrices(X, y):
    """
    Precomputes relevance (MI with target) and redundancy (MI between features)
    matrices with optimized data types and function calls.
    """
    n_samples, n_features = X.shape

    # --- Setup ---
    n_states_X = np.zeros(n_features, dtype=np.int32)
    for i in prange(n_features):
        n_states_X[i] = int(np.max(X[:, i])) + 1
    
    n_states_y = int(np.max(y)) + 1
    y_f32 = y.astype(np.float32)

    # --- Stage 1: Relevance Calculation ---
    relevance_scores = np.zeros(n_features, dtype=np.float32)
    for i in prange(n_features):
        relevance_scores[i] = _calculate_mi_optimized(
            X[:, i], n_states_X[i], y_f32, n_states_y, n_samples
        )

    # --- Stage 2: Redundancy Calculation ---
    redundancy_matrix = np.zeros((n_features, n_features), dtype=np.float32)
    for i in prange(n_features):
        for j in range(i + 1, n_features):
            mi = _calculate_mi_optimized(
                X[:, i], n_states_X[i], X[:, j], n_states_X[j], n_samples
            )
            redundancy_matrix[i, j] = mi
            redundancy_matrix[j, i] = mi

    return relevance_scores, redundancy_matrix

def fs_mrmr(X, y, k):
    """
    Selects top k features using mRMR, now generalized for ANY discrete data.
    This function handles non-contiguous, negative, or large integer categories
    by mapping them to zero-based integers before computation.
    
    Args:
        X (np.ndarray): Feature data (n_samples, n_features). Can contain any discrete values.
        y (np.ndarray): Target labels. Can contain any discrete values.
        k (int): The number of top features to select.
        
    Returns:
        np.ndarray: A sorted array of the top k feature indices.
    """
    n_samples, n_features = X.shape
    if not (0 < k <= n_features):
        raise ValueError("k must be a positive integer less than or equal to the number of features.")

    unique_values = np.unique(np.concatenate([X.flatten(), y]))
    value_to_int = {value: i for i, value in enumerate(unique_values)}

    mapper = np.vectorize(value_to_int.get)
    X_encoded = mapper(X)
    y_encoded = mapper(y)
    
    # Optimize data types later, numba does not like when you can pass in many different types
    X_for_numba = X_encoded.astype(np.float32)
    y_for_numba = y_encoded.astype(np.int32)
    
    relevance, redundancy = _precompute_mi_matrices(X_for_numba, y_for_numba)
    selected_features = []

    remaining_mask = np.ones(n_features, dtype=bool)

    # Select the first feature: the one with the highest relevance.
    first_feature_idx = np.argmax(relevance)
    selected_features.append(first_feature_idx)
    remaining_mask[first_feature_idx] = False

    for _ in range(k - 1):
        best_score = -np.inf
        best_feature = -1
        
        # Get the indices of features that are still available.
        remaining_indices = np.where(remaining_mask)[0]

        for candidate_idx in remaining_indices:
            # Relevance term: I(candidate; y)
            relevance_score = relevance[candidate_idx]
            
            # Redundancy term: Σ I(candidate; selected) / |S|
            # Look up pre-computed MI values. This is extremely fast.
            redundancy_score = np.sum(redundancy[candidate_idx, selected_features])
            avg_redundancy = redundancy_score / len(selected_features)
            
            # mRMR score (MID formulation: Relevance - Redundancy)
            mrmr_score = relevance_score - avg_redundancy

            if mrmr_score > best_score:
                best_score = mrmr_score
                best_feature = candidate_idx

        # Add the best feature found and update the mask
        if best_feature != -1:
            selected_features.append(best_feature)
            remaining_mask[best_feature] = False
    print("done")
    return np.sort(np.array(selected_features))

@njit(parallel=True, cache=True)
def _precompute_redundancy_matrix(X):
    n_samples, n_features = X.shape
    redundancy_matrix = np.zeros((n_features, n_features), dtype=np.float32)
    for i in prange(n_features):
        for j in range(i + 1, n_features):
            mi = _calculate_mi(X[:, i], X[:, j], n_samples)
            redundancy_matrix[i, j] = mi
            redundancy_matrix[j, i] = mi
    return redundancy_matrix

def fs_chi2_rmr(X, y, k):
    """
    Selects top k features using a hybrid Chi2-RMR algorithm, now generalized
    for ANY discrete data.
    
    This function handles non-contiguous, negative, or large integer categories
    by mapping them to zero-based integers before computation.
    
    Args:
        X (np.ndarray): Feature data (n_samples, n_features). Can contain any discrete values.
        y (np.ndarray): Target labels. Can contain any discrete values.
        k (int): The number of top features to select.
        
    Returns:
        np.ndarray: A sorted array of the top k feature indices.
    """
    n_samples, n_features = X.shape
    if k > n_features:
        raise ValueError("k cannot be greater than the number of features.")

    all_values = np.concatenate([X.flatten(), y.flatten()])
    unique_values = np.unique(all_values)
    value_to_int = {value: i for i, value in enumerate(unique_values)}
    
    dtype = np.min_scalar_type(len(unique_values) - 1)
    X_encoded = np.empty(X.shape, dtype=dtype)
    for i in range(n_features):
        for j in range(n_samples):
            X_encoded[j, i] = value_to_int[X[j, i]]

    y_encoded = np.empty(y.shape, dtype=dtype)
    for i in range(n_samples):
        y_encoded[i] = value_to_int[y[i]]

    
    relevance_scores, _ = chi2(X_encoded, y_encoded)
    
    redundancy_matrix = _precompute_redundancy_matrix(X_encoded)
    
    selected_features = []
    remaining_features = list(range(n_features))
    
    first_feature_idx = np.argmax(relevance_scores)
    selected_features.append(first_feature_idx)
    remaining_features.remove(first_feature_idx)

    for _ in range(k - 1):
        if not remaining_features: break
        redundancy_scores = np.sum(redundancy_matrix[remaining_features][:, selected_features], axis=1)
        avg_redundancy = redundancy_scores / len(selected_features)
        mrmr_scores = relevance_scores[remaining_features] - avg_redundancy
        best_idx_in_remaining = np.argmax(mrmr_scores)
        best_feature = remaining_features.pop(best_idx_in_remaining)
        selected_features.append(best_feature)
        
    return np.sort(np.array(selected_features))



def generate_synthetic_data(n_samples, n_features, n_informative):
    """Generates synthetic discrete data for benchmarking."""
    X = np.random.randint(0, 3, size=(n_samples, n_features)).astype(np.int8)
    informative_indices = np.arange(n_informative)
    y_raw = np.sum(X[:, informative_indices], axis=1)
    y = (y_raw > np.median(y_raw)).astype(np.int8)
    if n_features > n_informative: X[:, n_informative] = X[:, 0]
    if n_features > n_informative + 1: X[:, n_informative + 1] = (X[:, 1] + np.random.randint(0, 2, n_samples)) % 3
    return X, y

def jaccard_similarity(set1, set2):
    """Calculates the Jaccard Index for two sets."""
    return len(set1.intersection(set2)) / len(set1.union(set2))

def main_benchmark_and_verify():
    """Main function to run the benchmark and print results."""
    try:
        import mrmr
        import pymrmr
        from feature_engine.selection import MRMR
    except ImportError as e:
        print(f"Error importing a library. Please run 'pip install mrmr-selection pymrmr skfeature-chappers feature_engine'.\nOriginal error: {e}")
        return

    benchmark_params = [
        (1000, 200, 100),
        (200, 2000, 100), # Uncomment for a more intensive test
    ]
    
    # Warm up Numba
    print("--- Warming up Numba JIT compilers ---")
    X_warmup, y_warmup = generate_synthetic_data(100, 50, 20)
    _ = fs_mrmr(X_warmup, y_warmup, k=10)
    #_ = fs_chi2_rmr(X_warmup, y_warmup, k=10)
    print("--- Warm-up complete. ---\n")

    for n_samples, n_features, k in benchmark_params:
        print("="*80)
        print(f"RUNNING: {n_samples} samples, {n_features} features, selecting {k}")
        print("="*80)
        
        X_np, y_np = generate_synthetic_data(n_samples, n_features, n_informative=int(k/2))
        X_pd = pd.DataFrame(X_np, columns=[f'f{i}' for i in range(n_features)])
        
        results_time = {}
        selected_features = {}
        
        print("--- Timing Execution ---")
        
        def run_and_log(name, func, *args):
            start = time.perf_counter()
            features = func(*args)
            duration = time.perf_counter() - start
            results_time[name] = duration
            selected_features[name] = features
            print(f"{name:<28}: {duration:.4f} s")

        run_and_log('Your mRMR (Numba)', fs_mrmr, X_np, y_np, k)
        #run_and_log('Your Chi2-RMR (Numba)', fs_chi2_rmr, X_np, y_np, k)
        
        # mrmr-selection
        run_and_log('mrmr-selection (C++)', lambda: [int(name[1:]) for name in mrmr.mrmr_classif(X=X_pd, y=y_np, K=k)])
        
        '''# pymrmr
        pymrmr_df = X_pd.copy(); pymrmr_df['target'] = y_np
        run_and_log('pymrmr (C)', lambda: [int(name[1:]) for name in pymrmr.mRMR(pymrmr_df, 'MID', k)])
'''
        
        # Feature-engine
        def run_feature_engine_mid():
            fe_mrmr = MRMR(variables=None, max_features=k, method="MID", discrete_features=True, regression=False)
            fe_mrmr.fit(X_pd, y_np)
            all_cols = set(range(n_features))
            dropped_cols = set(int(name[1:]) for name in fe_mrmr.features_to_drop_)
            return list(all_cols - dropped_cols)
        run_and_log('Feature-engine (MID)', run_feature_engine_mid)

        # --- Verification Step ---
        print("\n--- Verifying Feature Set Similarity (Jaccard Index) ---")
        
        method_names = list(selected_features.keys())
        similarity_matrix = pd.DataFrame(np.eye(len(method_names)), index=method_names, columns=method_names)

        for (method1, features1), (method2, features2) in combinations(selected_features.items(), 2):
            set1 = set(features1); set2 = set(features2)
            similarity = jaccard_similarity(set1, set2)
            similarity_matrix.loc[method1, method2] = similarity
            similarity_matrix.loc[method2, method1] = similarity
            
        print(similarity_matrix.to_string(float_format="%.3f"))
        print("\n")

if __name__ == "__main__":
    main_benchmark_and_verify()


/home/galynch/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


--- Warming up Numba JIT compilers ---
done
--- Warm-up complete. ---

RUNNING: 1000 samples, 200 features, selecting 100
--- Timing Execution ---
done
Your mRMR (Numba)           : 0.1931 s


100%|██████████| 100/100 [00:03<00:00, 28.88it/s]


mrmr-selection (C++)        : 4.7276 s
Feature-engine (MID)        : 35.5496 s

--- Verifying Feature Set Similarity (Jaccard Index) ---
                      Your mRMR (Numba)  mrmr-selection (C++)  Feature-engine (MID)
Your mRMR (Numba)                 1.000                 0.626                 0.639
mrmr-selection (C++)              0.626                 1.000                 0.613
Feature-engine (MID)              0.639                 0.613                 1.000


RUNNING: 200 samples, 2000 features, selecting 100
--- Timing Execution ---
done
Your mRMR (Numba)           : 1.3164 s


100%|██████████| 100/100 [00:06<00:00, 14.34it/s]


mrmr-selection (C++)        : 7.1597 s
Feature-engine (MID)        : 134.0680 s

--- Verifying Feature Set Similarity (Jaccard Index) ---
                      Your mRMR (Numba)  mrmr-selection (C++)  Feature-engine (MID)
Your mRMR (Numba)                 1.000                 0.361                 0.639
mrmr-selection (C++)              0.361                 1.000                 0.325
Feature-engine (MID)              0.639                 0.325                 1.000




In [2]:
!git add .
!git commit -m "editing mrmr"
!git push

[main b502a7a] editing mrmr
 1 file changed, 66 insertions(+), 22 deletions(-)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 20 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 1.01 KiB | 515.00 KiB/s, done.
Total 5 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/GavinLynch04/FastRelief.git
   eaccd18..b502a7a  main -> main
